IMPORTS & PREPRODUCTION

In [1]:
import time

In [2]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
from numba import cuda

In [4]:
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [5]:
import wandb
wandb.login()
wandb.init(project="LeNet_5_test", entity="hyunseokerikjung")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hyunseokerikjung. Use `wandb login --relogin` to force relogin


In [6]:
from wandb.keras import WandbCallback

Data Processing

In [7]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [8]:
aug_df = pd.read_csv('augmented_emnist.csv')
aug_df.rename(columns={'25':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

train_y = np.array(aug_df.pop('Target'))
train_x = aug_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [8]:
train_df = pd.read_csv('emnist-byclass-train.csv/emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

train_y = np.array(train_df.pop('Target'))
train_x = train_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [9]:
test_df = pd.read_csv('emnist-byclass-test.csv/emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)

In [10]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)

In [11]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode="constant",
    cval=0
)

MODELS

Original LeNet_5

In [11]:
lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])
lenet_5_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [12]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_model.evaluate(test_x, test_y)

Epoch 1/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.5816 - accuracy: 0.8079 - val_loss: 0.4674 - val_accuracy: 0.8347
Epoch 2/100
17449/17449 [==============================] - 54s 3ms/step - loss: 0.4497 - accuracy: 0.8407 - val_loss: 0.4576 - val_accuracy: 0.8392
Epoch 3/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4319 - accuracy: 0.8453 - val_loss: 0.4496 - val_accuracy: 0.8397
Epoch 4/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4220 - accuracy: 0.8480 - val_loss: 0.4377 - val_accuracy: 0.8444
Epoch 5/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4159 - accuracy: 0.8497 - val_loss: 0.4338 - val_accuracy: 0.8471
Epoch 6/100
17449/17449 [==============================] - 58s 3ms/step - loss: 0.4113 - accuracy: 0.8510 - val_loss: 0.4303 - val_accuracy: 0.8467
Epoch 7/100
17449/17449 [==============================] - 57s 3ms/step - loss: 0.4067 - accuracy: 0.8528 - val_

[0.41677936911582947, 0.851464033126831]

LeNet_5_SeLU

SeLU performs better(speed & accuracy)

In [13]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [14]:
lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_selu_model.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_selu_model.evaluate(test_x, test_y)

Epoch 1/10
17449/17449 [==============================] - 51s 3ms/step - loss: 0.5755 - accuracy: 0.8070 - val_loss: 0.4708 - val_accuracy: 0.8314
Epoch 2/10
17449/17449 [==============================] - 50s 3ms/step - loss: 0.4502 - accuracy: 0.8397 - val_loss: 0.4465 - val_accuracy: 0.8389
Epoch 3/10
17449/17449 [==============================] - 51s 3ms/step - loss: 0.4300 - accuracy: 0.8452 - val_loss: 0.4342 - val_accuracy: 0.8444
Epoch 4/10
17449/17449 [==============================] - 50s 3ms/step - loss: 0.4200 - accuracy: 0.8479 - val_loss: 0.4274 - val_accuracy: 0.8462
Epoch 5/10
17449/17449 [==============================] - 50s 3ms/step - loss: 0.4120 - accuracy: 0.8502 - val_loss: 0.4326 - val_accuracy: 0.8424
Epoch 6/10
17449/17449 [==============================] - 51s 3ms/step - loss: 0.4065 - accuracy: 0.8521 - val_loss: 0.4373 - val_accuracy: 0.8455
Epoch 7/10
17449/17449 [==============================] - 50s 3ms/step - loss: 0.4026 - accuracy: 0.8527 - val_loss: 0

[0.4332740604877472, 0.8460480570793152]

He Yanmei SeLU Lenet_5 model

Lowers amount of parameters = lower inference time, less stable model

In [11]:
keras.backend.clear_session()

lenet_5_improved_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=2, activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.Conv2D(64, kernel_size=3, strides=2, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Conv2D(32, kernel_size=1, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

In [12]:
lenet_5_improved_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_improved_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_improved_model.evaluate(test_x, test_y)

Epoch 1/100
17449/17449 [==============================] - 55s 3ms/step - loss: 1.0739 - accuracy: 0.6858 - val_loss: 0.7372 - val_accuracy: 0.7655
Epoch 2/100
17449/17449 [==============================] - 50s 3ms/step - loss: 0.6588 - accuracy: 0.7867 - val_loss: 0.6335 - val_accuracy: 0.7957
Epoch 3/100
17449/17449 [==============================] - 51s 3ms/step - loss: 0.5938 - accuracy: 0.8041 - val_loss: 0.5728 - val_accuracy: 0.8088
Epoch 4/100
17449/17449 [==============================] - 51s 3ms/step - loss: 0.5607 - accuracy: 0.8134 - val_loss: 0.5567 - val_accuracy: 0.8149
Epoch 5/100
17449/17449 [==============================] - 50s 3ms/step - loss: 0.5389 - accuracy: 0.8193 - val_loss: 0.5499 - val_accuracy: 0.8155
Epoch 6/100
17449/17449 [==============================] - 50s 3ms/step - loss: 0.5246 - accuracy: 0.8234 - val_loss: 0.5298 - val_accuracy: 0.8224
Epoch 7/100
17449/17449 [==============================] - 50s 3ms/step - loss: 0.5135 - accuracy: 0.8267 - val_

[0.455183744430542, 0.8441395163536072]

Build #20220515

87.09% accuracy

In [ ]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.summary()

In [ ]:
my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
my_lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

my_lenet_5_selu_model.evaluate(test_x, test_y)

In [ ]:
now = time.time()
now = time.localtime(now)
my_lenet_5_selu_model.save("my_lenet5_emnist_model_%2d%02d%02d.h5" %(now.tm_year, now.tm_mon, now.tm_mday), save_format="h5")

Build #20220517

testing wandb

In [ ]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.summary()

In [ ]:
lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

lenet_5_selu_model.evaluate(test_x, test_y)

In [11]:
def sweep_train(config_defaults=None):
    
    # Set default values
    config_defaults = {
      'n_first_node' : 32,
      'n_second_node' : 128,
      'n_third_node' : 64,
      'n_dense_size' : 128
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # initialize model
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(wandb.config.n_first_node, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(wandb.config.n_second_node, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(wandb.config.n_third_node, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(62, activation='softmax'))

    print(model.summary())

    # Instantiate an optimizer to train the model.
    optimizer = keras.optimizers.Adam(learning_rate=3e-4, beta_1=0.9, beta_2=0.999)
    # Instantiate a loss function.
    # loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    hist = model.fit(val_x, val_y, epochs=10, validation_data=(test_x, test_y), callbacks=[WandbCallback()])

In [12]:
sweep_config = {
  'method': 'random',
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
  'parameters': {
      'n_first_node' : {
          'values' : [32, 64, 128]
      },
      'n_second_node' : {
          'values' : [128, 256]
      },
      'n_third_node' : {
          'values' : [64, 128, 256]
      }
  }
}

sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=sweep_train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: fd73n8t8
Sweep URL: https://wandb.ai/hyunseokerikjung/uncategorized/sweeps/fd73n8t8


wandb: Agent Starting Run: z6mv1ujh with config:
wandb: 	n_first_node: 64
wandb: 	n_second_node: 256
wandb: 	n_third_node: 128


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 256)       409856    
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 128)       295040    
                                                                 
 batch_normalization_2 (Batc  (None, 22, 22, 128)      5

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


...

wandb: Agent Starting Run: p4rtpee0 with config:
wandb: 	n_first_node: 32
wandb: 	n_second_node: 128
wandb: 	n_third_node: 256


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       102528    
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 128)      512       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 256)       295168    
                                                                 
 batch_normalization_2 (Batc  (None, 22, 22, 256)      1

accuracy,▁▆▇▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
val_accuracy,▁▄▅▅▇▇█▆▇▇
val_loss,█▅▄▄▂▂▁▂▂▁
accuracy,0.86556
best_epoch,6
best_val_loss,0.3969
epoch,9
loss,0.366
val_accuracy,0.85433


wandb: Agent Starting Run: v09raihf with config:
wandb: 	n_first_node: 32
wandb: 	n_second_node: 128
wandb: 	n_third_node: 128


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       102528    
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 128)      512       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 128)       147584    
                                                                 
 batch_normalization_2 (Batc  (None, 22, 22, 128)      5

BUILD #220523

OH GOD PLEASE WORK

In [13]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(128, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(512, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 128)       3328      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 256)       819456    
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 512)       1180160   
                                                                 
 batch_normalization_2 (Batc  (None, 22, 22, 512)      2

In [13]:
my_lenet_5_selu_model.evaluate(test_x, test_y)

3636/3636 [==============================] - 22s 6ms/step - loss: 0.3583 - accuracy: 0.8701


[0.35832297801971436, 0.8700761795043945]

In [14]:
my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)
  
my_lenet_5_selu_model.fit(train_x, train_y, epochs=240, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, WandbCallback()])

my_lenet_5_selu_model.evaluate(test_x, test_y)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/240
17449/17449 [==============================] - 250s 14ms/step - loss: 0.5072 - accuracy: 0.8303 - val_loss: 0.4369 - val_accuracy: 0.8474 - _timestamp: 1654999508.0000 - _runtime: 3618.0000
Epoch 2/240
17449/17449 [==============================] - 250s 14ms/step - loss: 0.3931 - accuracy: 0.8587 - val_loss: 0.3994 - val_accuracy: 0.8545 - _timestamp: 1654999758.0000 - _runtime: 3868.0000
Epoch 3/240
17449/17449 [==============================] - 251s 14ms/step - loss: 0.3731 - accuracy: 0.8635 - val_loss: 0.3747 - val_accuracy: 0.8649 - _timestamp: 1655000009.0000 - _runtime: 4119.0000
Epoch 4/240
17449/17449 [==============================] - 251s 14ms/step - loss: 0.3605 - accuracy: 0.8676 - val_loss: 0.3763 - val_accuracy: 0.8604 - _timestamp: 1655000260.0000 - _runtime: 4370.0000
Epoch 5/240
17449/17449 [==============================] - 252s 14ms/step - loss: 0.3512 - accuracy: 0.8700 - val_loss: 0.3569 - val_accuracy: 0.8699 - _timestamp: 1655000512.0000 - _runtime: 

[0.34490063786506653, 0.8726896047592163]

In [15]:
now = time.time()
now = time.localtime(now)
my_lenet_5_selu_model.save("my_lenet5_emnist_model_%2d%02d%02d.h5" %(now.tm_year, now.tm_mon, now.tm_mday), save_format="h5")